## ProjF2 - Data Exploration

Use this document as a template to provide some exploratory analysis of your data similar to the one in ProjC1. You are welcome to go in as much depth as needed.

Make sure you keep the sections specified in this template, but you are welcome to add more cells with your code or explanation as needed.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### 1. Load and Visualize Data

Let's start by loading your entire dataset or a sample if it is a larger dataset.

In [147]:
import mat73
# mat = mat73.loadmat('/Users/mayur/Downloads/20200627_0942.mat')



import scipy.io
import numpy as np
import matplotlib.pyplot as plt


import py7zr

archive = py7zr.SevenZipFile('/Users/mayur/Downloads/20200706_1015.zstd.7z', mode='r')
archive.extractall(path="/Users/mayur/Downloads/")
archive.close()
# Load the MATLAB file
data = mat73.loadmat('/Users/mayur/Downloads/20200706_1015.mat')
print(data)
# x = data['x']
# y = data['y']
lambda_val = data['lambda']
s0 = data['S0int']
s1 = data['S1int']
s2 = data['S2int']
# print(s)

# Extract a specific wavelength slice (e.g., the first wavelength)
s0_slice = s0[:,:,0]
print(s0.shape)



{'Nframes': array(2900.), 'S0int': array([[[21919, 21642, 20554, ...,     0,     0,     0],
        [21413, 21861, 20329, ...,     0,     0,     0],
        [22039, 23091, 21793, ...,     0,     0,     0],
        ...,
        [ 5341,  5212,  5434, ...,     0,     0,     0],
        [ 5119,  4639,  4359, ...,     0,     0,     0],
        [12117, 12314, 12932, ...,     0,     0,     0]],

       [[21801, 21084, 19981, ...,     0,     0,     0],
        [21320, 21587, 20195, ...,     0,     0,     0],
        [22607, 22912, 21069, ...,     0,     0,     0],
        ...,
        [ 5270,  5126,  5394, ...,     0,     0,     0],
        [ 4801,  4611,  4536, ...,     0,     0,     0],
        [ 9896, 10632, 10685, ...,     0,     0,     0]],

       [[21829, 20500, 19987, ...,     0,     0,     0],
        [22243, 21396, 20173, ...,     0,     0,     0],
        [22891, 22184, 20725, ...,     0,     0,     0],
        ...,
        [ 5099,  5096,  5078, ...,     0,     0,     0],
        [ 

In [144]:
s0 , s1, s2 = s0[800:900,600:700,55],s1[800:900,600:700,55],s2[800:900,600:700,55]
LL=np.zeros([100,100])
DOLP = np.zeros([100,100])


S0_normalized = np.ones_like(s0)

s_n = np.sqrt(s1**2 + s2**2)

# Normalize S1, S2, S3
s1_norm = s1 / s_n
s2_norm = s2 / s_n
# s0_norm = s0 / s0

# Clip the normalized values to ensure they fall within the range [-1, 1]
s1_normalized = np.clip(s1_norm, -1, 1)
s2_normalized = np.clip(s2_norm, -1, 1)


condition_met = (S0_normalized**2 >= s1_normalized**2 + s2_normalized**2 ).all()

# Display the result
if condition_met:
    print("Condition s02 ≥ s12 + s22 + s32 is met.")
else:
    print("Condition s02 ≥ s12 + s22 + s32 is not met.")
print(s0.shape)

Condition s02 ≥ s12 + s22 + s32 is not met.
(100, 100)


/var/folders/8t/cg46ygc52xq8ts0rysyjhfn40000gn/T/ipykernel_3561/3135231120.py:8: RuntimeWarning: invalid value encountered in sqrt
  s_n = np.sqrt(s1**2 + s2**2)


In [145]:
# s0_position = s0[0,0,:]
#
# # Calculate mean intensity across all wavelengths at each spatial position
# mean_intensity = np.mean(s0, axis=2)
#
# Visualize a specific wavelength slice
# plt.imshow(s0_slice)
# plt.colorbar()
# plt.title('Specific Wavelength Slice')
# plt.show()
# print(s1.shape)

# s0_normalized = np.clip(s0_norm, -1, 1)




for i in range(100):
    for j in range(100):
        if ~np.isnan(s1[i][j].all()) and ~np.isnan(s2[i][j].all()) and ~np.isnan(s0[i][j].all()) and s0[i][j]**2 >= s1[i][j]**2 + s2[i][j]**2:
                LL[i][j] =np.sqrt(s1[i][j]**2+s2[i][j]**2)
                DOLP[i][j] = LL[i][j]/s0[i][j]


print(LL.shape,"shapee",s0.shape)

# print(s2[~np.isnan(s2)]**2,"555555")
# V = LL[~np.isnan(LL)]
# print(V,"**************")

# Calculate angle of polarization (AOP)
print(DOLP[~np.isnan(DOLP)].shape)
DOLP = DOLP[~np.isnan(DOLP)]

(100, 100) shapee (100, 100)
(10000,)


In [146]:
# DOLP = DOLP[~np.isnan(DOLP)]
AOP = 0.5 * np.arctan2(s0, s1)
print(DOLP.shape)

circular_threshold = 0.99  # Adjust as needed based on your requirements

# Calculate statistics for DOLP and AOP across the entire image
mean_DOLP = np.mean(DOLP)
mean_AOP = np.mean(AOP)
print(mean_DOLP,"--",mean_AOP)
polarization_state = ""
if mean_DOLP <= circular_threshold:
    # Circular polarization
    polarization_state = "Circular"
elif mean_DOLP < 1:
    # Linear or elliptical polarizat\ion
    polarization_state = "Linear or Elliptical"

# Print the predominant polarization state
print("Predominant polarization state:", polarization_state)



(10000,)
0.1953764616599019 -- 0.38459736
Predominant polarization state: Linear or Elliptical


In [80]:
print(np.amax(s2))
# LL = np.sqrt(s1[~np.isnan(s1)]**2+ s2[~np.isnan(s2)]**2)
# # print(s2[~np.isnan(s2)]**2,"555555")
# V = LL[~np.isnan(LL)]
# print(V)
# # DOLP = V / s0[~np.isnan(s0)]
# mean_DOLP = np.mean(DOLP)
# # mean_AOP = np.mean(AOP)
# print(mean_DOLP)

1048


In [13]:
data = mat73.loadmat('/Users/mayur/Downloads/20200706_1015.mat')

lambda_val = data['lambda']
s0 = data['S0int']
s1 = data['S1int']
s2 = data['S2int']
# print(s)


print(DOLP.shape)
print(DOLP)
print(s1.shape)

KeyError: 'x'

Provide some visualization of the data. For example,
- If your data is time series, you can plot a sequence and the response or label.
- If tabular data, you can plot histograms of the data.
- If working with image recognition or segmentation, you can show an image and the corresponding output.
- If it is a video that you are processing, just show screenshots.
- If doing something with reinforcement learning, you can provide samples of the simulation environment used.

In [3]:
### YOUR CODE HERE

### 2. Compute Some Statistics from your Data

First, extract some characterisitics about the input (e.g., image size, means and standard deviation of values, sample frequency).

In [4]:
### YOUR CODE HERE

Next, provide some characteristics about your output (e.g., histograms, spatial distribution of bounding boxes in the case of object detection, etc.). In the case of cases such as tasks such as reinforcement learning, you can provide details about the desired output and rewards used. You can show some correlations between inputs and outputs if possible.

In [5]:
### YOUR CODE HERE

### 3. Comment on Your Data

Answer these questions as appropriate?
- Is your data balanced?
- Are there disparities in your input data? Does it show some specific patterns?

`### Comment here`